HELIC 1x association report ( Aug./Sept. 15)
===========================

This report contains the protocol for reproducing the MANOLIS and POMAK 1x association studies.

## Phenotype preparation

Two sets of phenotypes were produced: a "brainless" one that mimicks UK10K transformations based primarily on inverse normal transforms, arbitrary thresholds and uncontrolled regression by $age$ and $age^2$, and a second one that hand-curated every phenotype in an attempt to find an invertible transformation that would best normalise the data.

### UK10K-style transformations

#### MANOLIS
`6.bioch.final.txt` (a restriction of Rachel's QC-ed file `~rm18/Rotation_1/MANOLIS/New_Phenotypes/MANOLIS.new.phenotypes.removal1.txt` to the 1239 4x1x individuals) was merged with `ferritin.pheno` (extracted with the PhenotypeDB tool) into `/nfs/t144_helic/Phenotypes/Arthur/source_file/HA`, then merged with leptin and adiponectin from Rachel's directory, with the blood phenotypes extracted from the DB, and finally with the master extract from `Phenotypes/DataExtracts/WGS`.

Total $n=51$ phenotypes excluding sex and age in file `DataExtract.x.ferritin.x.bioch.x.elisa.phenotypes`.

### Pomak
The same, except `/nfs/t144_helic/Phenotypes/OriginalFiles/biochemicals_HP.txt` was used for the biochemicals, and ELISA measurements were not available for this cohort.


Total $n=48$.

### Invertible continuous transformations


## Genotype preparation

We extracted monmorphics and converted to Binary Plink format using:
>`for i in {1..22}; do echo /software/team144/plink-versions/beta3r/plink --vcf /nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-secondpass/\$i.vcf.gz --make-bed --out \$i.nomono --memory 5000 --maf 0.0003; done | ~/array 5g vcf2plink`

Then missing SNP names had to be built (to chromosome:position).

>`for i in {1..22}; do echo ./snpnames.sh $i.nomono.bim; done | ~/array 5g snpnames`

For every chromosome, `~/assoc_by_chunk.pl` was called into the `assoc_chunks` subdir to generate input chunks.
The same procedure was repeated in `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015_POMAK`.

## Genetic Relatedness Matrix calculation

In order to compute the GRM we first generate a genomewide input file in `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015/matrices/general_input`, using:

>`hsub 10g /software/team144/plink-versions/beta3v/plink --bfile 1.nomono --make-bed --memory 9500 --merge-list bed.list --out matrices/general_input/4x1xseq`

First we used that file for calculating a matrix with all variants in the `unfiltered` directory.
Then, we pruned using `plink --indep 50 5 2` into `ldprune_indep_50_5_2`, finally we removed HWE failed variants with `--hwe 1e-5 midp` in `ldprune_indep_50_5_2.hwe.1e-5`.

Only the last combination was calculated for the POMAK, in `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015_POMAK/matrices/ldprune_indep_50_5_2.hwe.1e-5`.

## Association pipeline

The necessary steps are:
* attribute phenotype to input chunks;
* associate each chunk with GEMMA using precomputed relatedness matrix;
* merge all chunks;
* filter for MAF>0.001 ($0.001*1239=1.2$), removing singletons;
* produce QQ and Manhattan plots for this filtered dataset;
* GC-correct the filtered dataset;
* produce plots for that dataset.

### Pomak

All of the above steps are implemented in [`associate-POMAK-residualised.sh`](associate-POMAK-residualised.sh). The script was run the following way:
> `ls /nfs/t144_helic/Phenotypes/Arthur/source_file/HP/Residuals-Plots/ | sed 's/_HP.WGS.UK10K.*//g' | sort -u | while read line; do echo ~/association-scripts/associate-POMAK-residualised.sh \$line;  done | ~/array 2g assoc_pomak`
in `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015_POMAK/output/ldprune_indep_50_5_2.hwe.1e-5`.

### MANOLIS
Everything was run in a very similar way (the paths were slightly different when this was done):

>`/nfs/t144_helic/Phenotypes/Arthur/source_file/Residuals-Plots/ | sed 's/_HA.WGS.UK10K.*//g' | tail -n+3 | sort -u | while read line; do echo ~/association-scripts/associate-MANOLIS-residualised.sh $line;  done | ~/array 2g assoc_manol`

In MANOLIS, which was run first, the association script did not yet implement the GC-correction step ([`associate-MANOLIS-residualised.sh`](associate-MANOLIS-residualised.sh)), so the GC-correction script had to be run separately afterwards, like this:

>`find -type f -iname "*0.001.assoc.txt.gz" | while read line; do echo ~/association-scripts/gccorrect.sh $1; done | array 5g gc_correct`

Some drawing commands might fail because of out of memory errors. In which case, do:

>`find -type f -iname "*0.001*gz" | sed 's/^..//;s/\/.*//' | while read trait; do if [ ! -e $trait/*0.001*qq* ]; then cd $trait; phenotype=$trait; hsub 10g -I -q yesterday ~/man_qq_annotate --chr-col chr --pos-col ps --auto-label --pval-col p_score --title "MANOLIS-$phenotype-MAF0.001" --sig-thresh 1e-08 --sig-thresh-line 1e-08 MANOLIS.$phenotype.maf0.001.assoc.txt MANOLIS.$phenotype.maf0.001.assoc; fi; done`

### Both: Specially transformed phenotypes

 > `find /nfs/t144_helic/Phenotypes/Arthur/HA_newtransforms/ -maxdepth 1 -type d| sed 's/.*\///' |tail -n+2 | while read line; do echo ~/association-scripts/associate-both-specialPhenotypes.sh $line; done | ~/array 10g assoc_c`
 
 There is no need to run anything else as this script runs first MANOLIS, then POMAK.
 
## Meta-analysis

We get the intersecting phenotypes between both cohorts, and run the meta-analysis script that deals with conversion and runs GWAMA with GC-correction.

> `comm -1 -2 <(find ../assoc_freeze_Summer2015/output/ldprune_indep_50_5_2.hwe.1e-5/ -maxdepth 1 -type d | grep -v "custom" | tail -n+2 | sed 's/.*\///'| sort) <(find ../assoc_freeze_Summer2015_POMAK/output/ldprune_indep_50_5_2.hwe.1e-5/ -maxdepth 1 -type d | grep -v "custom" | tail -n+2 | sed 's/.*\///' | sort) | while read line; do echo /lustre/scratch114/projects/helic/meta_freeze_Summer2015/meta_helic.sh $line; done | ~/array 15g meta yesterday`

## Quality control measures

There is little we can do to filter variants post imputation (see [`randomWoods`](https://bitbucket.org/agilly/randomWoods) project). To assess what the quality and number of variants intercepted by our pipeline are, we summarize them on a single graph (all this in `/nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-secondpass/concordance`).

First, fetch concordance with genotypes using `concordance_with_genotypes2.pl` :

```bash
for i in {1..22}; do echo $i; ~/concordance_with_genotype2.pl ../$i.vcf.gz $i > concordance.$i; done
```

Then, we get the variants seen by the calling step, before imputation:]

```bash
for i in {1..22}; do echo $i; zgrep -v '^#' ../../VQSR-filtered/$i.hardfiltered.vcf.gz | cut -f1-2 > $i.calledpos; done
```

Finally, we merge:

```bash
for i in {1..22}; do join -1 1 -2 2 -a 1 <(sort -k1,1 concordance.$i) <(sort -k2,2 $i.calledpos) | awk -v chr=$i '{if(NF<11){$11=0}else{$11=1}print chr, $0}'; done > all.merged.concordance.calledpos 
```

Calling R to the rescue for the graphical stuff:

```r
library(data.table)
e=data.frame(fread("all.merged.concordance.calledpos"))
e=e[,2:12]
colnames(e)=paste(rep("V", 11), 1:11, sep="")
d=e
maf=0
library(colorspace)
# $$
maf[d$V9<=1239]=d[d$V9<=1239,]$V9/(2*1239)
maf[d$V9>1239]=(2*1239-d[d$V9>1239,]$V9)/(2*1239)
cutmaf=cut(maf, breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
cutmafna=cut(maf[d$V3!="NA"], breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
par(mar=c(5.1, 5.1, 4.1, 2.1))
b=barplot(table(cutmafna)/table(cutmaf), names=c("0.01-0.05%", "0.05-0.1%", "0.1-0.5%", "0.5-1%", "1-5%", "5-10%", "10-20%", "20-30%", "30-40%", "40-50%"), xlab="MAF bin", ylab="Genotype concordance", ylim=c(0,1.05), cex.axis=2, cex.names=1.5, cex.lab=2, font=2, font.lab=2, col=rev(sequential_hcl(11, h=120, l=c(30, 100), power=0.9))[2:11])
e=d
d=d[d$V11==1,]
maf=0
cutmafo=cutmaf
library(colorspace)
maf[d$V9<=1239]=d[d$V9<=1239,]$V9/(2*1239)
maf[d$V9>1239]=(2*1239-d[d$V9>1239,]$V9)/(2*1239)
cutmaf=cut(maf, breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
cutmafna=cut(maf[d$V3!="NA"], breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
par(mar=c(5.1, 5.1, 4.1, 2.1))
c=barplot(table(cutmafna)/table(cutmafo), names=c("0.01-0.05%", "0.05-0.1%", "0.1-0.5%", "0.5-1%", "1-5%", "5-10%", "10-20%", "20-30%", "30-40%", "40-50%"), xlab="MAF bin", ylab="Genotype concordance", ylim=c(0,1.05), cex.axis=2, cex.names=1.5, cex.lab=2, font=2, font.lab=2, col=rev(sequential_hcl(11, h=-90, l=c(30, 100), power=0.9))[2:11], add=T)
d=e
d=d[d$V10<10,]

maf=0
library(colorspace)
maf[d$V9<=1239]=d[d$V9<=1239,]$V9/(2*1239)
maf[d$V9>1239]=(2*1239-d[d$V9>1239,]$V9)/(2*1239)
cutmaf=cut(maf, breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
cutmafna=cut(maf[d$V3!="NA"], breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
#par(mar=c(5.1, 5.1, 4.1, 2.1))
#b=barplot(table(cutmafna)/table(orcutmaf), names=c("0.01-0.05%", "0.05-0.1%", "0.1-0.5%", "0.5-1%", "1-5%", "5-10%", "10-20%", "20-30%", "30-40%", "40-50%"), xlab="MAF bin", ylab="Genotype concordance", ylim=c(0,1.05), cex.axis=1.8, cex.names=1.8, cex.lab=2, col=rev(sequential_hcl(10, h=-90, l=c(30, 100), power=0.9)))
tan=1239*2
tgn=1239
ma=0
ma[d$V9<=tgn]=d[d$V9<=tgn,]$V8/d[d$V9<=tgn,]$V9
ma[d$V9>tgn]=d[d$V9>tgn,]$V7/(tan-d[d$V9>tgn,]$V9)
#points(x=b, y=, type='b', col="blue")
lenient=aggregate(ma, list(cutmaf), function(x)  mean(x, na.rm=T))[,2]
points(x=b, y=aggregate(d$V2/1239, list(cutmaf), function(x)  mean(x, na.rm=T))[,2], type='b', lwd=2)
mtext("Proportion of variants intercepted", side=4, cex.lab=1,las=0, cex=2)

axis(4, seq(0, 1, by=0.2), pos=12.1, cex.axis=1.8)
# $$
maf=0
ma=0
maf[d$V9<=1239]=d[d$V9<=1239,]$V9/(2*1239-d[d$V9<=1239,]$V10)
maf[d$V9>1239]=(2*1239-d[d$V9>1239,]$V10-d[d$V9>1239,]$V9)/(2*1239-d[d$V9>1239,]$V10)
#write.table(d[maf< & maf<,])
cutmaf=cut(maf, breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
cutmafna=cut(maf[!is.na(d$V3)], breaks=c(0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5), labels=0:9)
tan=1239*2
tgn=1239
ma=0
ma[d$V9<=tgn]=d[d$V9<=tgn,]$V8/d[d$V9<=tgn,]$V9
ma[d$V9>tgn]=d[d$V9>tgn,]$V7/(tan-d[d$V9>tgn,]$V9)
conservative=aggregate(ma, list(cutmaf), function(x)  mean(x, na.rm=T))[,2]
points(x=b, y=(lenient+conservative)/2, type='b', col="blue", lwd=2)


```
And here the result (not generated in ipython):

![plot](Rplots.png)

## Peak finding and plotting

Manhattand and QQ-plotting is taken care of by the association script. We want to produce RAPs around the peaks we see in the data, we also want to see our interactive bokeh plot for quick analysis of peaks. We run:
```
find -type d -maxdepth 1 | grep -v custom | sed 's/^..//' | tail -n+2 | while read line; do echo ./rungraphs.sh $line; done | ~/array 5g plots
```

## Peak investigation

* `~/association-scripts/query1x.sh` will display association statistics of a variant within both cohorts and the meta-analysis. To get the assoc and meta analysis data for a particular trait and variant, do `t=LYM;c=3;p=80088046;/nfs/users/nfs_a/ag15/association-scripts/query1x.sh $t $c $p | awk '{if(NR==1){p=$NF; b=$8; se=$9;}if(NR==2){mp=$12;eff=$NF}if(NR==3){pp=$NF;print $7}}END{OFS=" "; print p, b, se, pp,mp, eff}'`
* For getting the genotype calls for sequencing, refinement and phasing side by side, we do the following:
```bash
chr=3;ps=80088046; join -j1 <(join -j1 <(cat <(zgrep -m 1 CHROM /nfs/t144_helic/sequencing-data/HA-4x1x-seq/VQSR-filtered/$chr.hardfiltered.vcf.gz) <(tabix /nfs/t144_helic/sequencing-data/HA-4x1x-seq/VQSR-filtered/$chr.hardfiltered.vcf.gz $chr:${ps}-$ps) | ~/transpose-2.0/src/transpose -l 2000x2000 -t -f 200| grep -v NA | awk 'OFS="\t"{if(NF<2){$(NF+1)="NA"}print}'| sed -f /nfs/t144_helic/sequencing-data/sample_tables/HA.sed | sed -f /nfs/t144_helic/sequencing-data/sample_tables/HA.duplicates.sed |sort -k1,1) <(cat <(zgrep -m 1 CHROM /nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-firstpass/$chr.vcf.gz) <(tabix /nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-firstpass/$chr.vcf.gz $chr:${ps}-$ps) | ~/transpose-2.0/src/transpose -l 2000x2000 -t -f 200 | grep -v NA | awk 'OFS="\t"{if(NF<2){$(NF+1)="NA"}print}'| sed -f /nfs/t144_helic/sequencing-data/sample_tables/HA.sed | sed -f /nfs/t144_helic/sequencing-data/sample_tables/HA.duplicates.sed |sort -k1,1)) <(cat <(zgrep -m 1 CHROM /nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-secondpass/$chr.vcf.gz) <(tabix /nfs/t144_helic/sequencing-data/HA-4x1x-seq/imputed-secondpass/$chr.vcf.gz $chr:${ps}-$ps) | ~/transpose-2.0/src/transpose -l 2000x2000 -t -f 200 | grep -v NA | sort -k1,1) | sort -k1,1 -r | tr ' ' '\t'  | grep -e '1|' -e '|1' -e '1/' -e '/1' -e CHROM -e POS -e FILTER -e INFO -e FORMAT -e REF -e ALT -e QUAL|less
```
* To associate using _only the sequence data_, do `~/association-scripts/associate_MANOLIS_in_sequence.sh RDW 11 7449221`
* another script that doesn't really work for now is `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015/output/ldprune_indep_50_5_2.hwe.1e-5/investigate_hit.sh`. It is supposed to query a curated bedfile of positive controls, but Daniel's file does not contain signals, but large regions around them.

### Pomaks

For pomaks we switched to a slightly more efficient way of generating the data in the summary excel table:

```bash
ls *html | sed 's/.peakda.*//' | while read i; do chr=$(echo $i |sed 's/\..*//');ps=$(echo $i |sed 's/.*\.//');pom_assoc=$(zgrep -w $ps *smart.gz);f_pom=$(echo $pom_assoc | cut -d' ' -f7); allele_string=$(echo $pom_assoc | cut -d' ' -f5)_$(echo $pom_assoc | cut -d' ' -f6);beta=$(echo $pom_assoc | cut -d' ' -f8);se=$(echo $pom_assoc | cut -d' ' -f9);p_pom=$(echo $pom_assoc | cut -d' ' -f16); trait=$(basename $(pwd));man_assoc=$(zgrep -w $ps ../../../../assoc_freeze_Summer2015/output/ldprune_indep_50_5_2.hwe.1e-5/$trait/*ct.gz); f_man=$(echo $man_assoc | cut -d' ' -f7); p_man=$(echo $man_assoc | cut -d' ' -f15);meta_assoc=$(zgrep -w $ps ../../../../meta_freeze_Summer2015/$trait/*gz); meta_p=$(echo $meta_assoc | cut -d' ' -f12);effects=$(echo $meta_assoc | awk '{print $NF}'); ensembl=$(perl /nfs/team144/ds26/FunctionalAnnotation/20150505_new_tool/VarAnnot.pl <(echo chr${chr}:${ps}-${ps}_$allele_string) 2> /dev/null | tail -n1); csq=$(echo $ensembl | awk -F ',' '{print $32}' | sed 's/\"//g'); freqs=$(echo $ensembl | cut -d"," -f40-44 |  sed 's/\"//g;');gene=$(echo $ensembl | cut -d',' -f10 | sed 's/\"//g');passoc=$(echo $ensembl | awk -F',' '{print $NF}'); seqp=$(~/association-scripts-git/associate_POMAK_in_sequence.sh $trait $chr $ps | tail -n1 | awk '{print $NF}');rs=$(echo $ensembl | cut -d"," -f5 |  sed 's/\"//g;');seq=$(vcf=/nfs/t144_helic/sequencing-data/HP-1x-seq/recalibration/recalibrated/$chr.recalibrated.vcf.gz;~/association-scripts-git/query_vcf.sh $vcf $chr $ps); imp=$(~/association-scripts-git/query_vcf.sh /nfs/t144_helic/sequencing-data/HP-1x-seq/imputed-firstpass/$chr.*vcf.gz $chr $ps| cut -d' ' -f3);phase=$(~/association-scripts-git/query_vcf.sh /nfs/t144_helic/sequencing-data/HP-1x-seq/imputed-secondpass-phased/$chr.reheaded.vcf.gz $chr $ps| cut -d' ' -f3);echo $trait,$chr,$ps,$rs,$f_pom,$f_man,$freqs,$csq,$gene,$passoc,$p_pom,$beta,$se,$p_man,$meta_p,$effects,$seqp,$seq $imp $phase; done
```
It automatically looks for p-values in MANOLIS as well but does not compute the meta-analysis p-value.

### Meta-analysis

```bash
ls *html | sed 's/.peakda.*//' | while read i; do chr=$(echo $i |sed 's/\..*//');ps=$(echo $i |sed 's/.*\.//'); trait=$(basename $(pwd));man_assoc=$(zgrep -w $ps ../../assoc_freeze_Summer2015/output/ldprune_indep_50_5_2.hwe.1e-5/$trait/*ct.gz | awk -v c=$chr '$1==c'); man_a1=$(echo $man_assoc | cut -d' '  -f5);man_a2=$(echo $man_assoc | cut -d' '  -f6); man_af=$(echo $man_assoc | cut -d' '  -f7);man_p=$(echo $man_assoc | cut -d' '  -f15);pom_assoc=$(zgrep -w $ps ../../assoc_freeze_Summer2015_POMAK/output/ldprune_indep_50_5_2.hwe.1e-5/$trait/*rt.gz| awk -v c=$chr '$1==c'); pom_a1=$(echo $pom_assoc | cut -d' '  -f5);pom_a2=$(echo $pom_assoc | cut -d' '  -f6); pom_af=$(echo $pom_assoc | cut -d' '  -f7);pom_p=$(echo $pom_assoc | cut -d' '  -f15); metassoc=$(zgrep -w $ps *.gz | awk -v c=$chr '$2==c');metaf=$(echo $metassoc | cut -d' ' -f6); allele_string=$(echo $metassoc | cut -d' ' -f4)_$(echo $metassoc | cut -d' ' -f5); meta_p=$(echo $metassoc | cut -d' ' -f12);effects=$(echo $metassoc | awk '{print $NF}'); ensembl=$(perl ~/VarAnnot.pl <(echo chr${chr}:${ps}-${ps}_$allele_string) 2> /dev/null | tail -n1); csq=$(echo $ensembl | awk -F ',' '{print $32}' | sed 's/\"//g'); freqs=$(echo $ensembl | cut -d"," -f40-44 |  sed 's/\"//g;');gene=$(echo $ensembl | cut -d',' -f15 | sed 's/\"//g');passoc=$(echo $ensembl | awk -F',' '{print $NF}');meta_p=$(echo $meta_assoc | cut -d' ' -f12);effects=$(echo $metassoc | awk '{print $NF}'); ensembl=$(perl ~/VarAnnot.pl <(echo chr${chr}:${ps}-${ps}_$allele_string) 2> /dev/null | tail -n1); csq=$(echo $ensembl | awk -F ',' '{print $32}' | sed 's/\"//g'); freqs=$(echo $ensembl | cut -d"," -f40-44 |  sed 's/\"//g;');gene=$(echo $ensembl | cut -d',' -f15 | sed 's/\"//g');passoc=$(echo $ensembl | awk -F',' '{print $NF}');echo $trait,$chr,$ps,$man_a1,$man_a2,$man_af,$man_p,$pom_a1,$pom_a2,$pom_af,$pom_p,$metaf,$meta_p,$effects,$gene,$freqs,$csq,$passoc;done
```

## Effective number of tests

### Effective number of traits

```
library(data.table)
ph=as.character(fread("phenotype.list", header=F)$V1)
m=data.table(ID=character())
for (i in ph){d=fread(paste(i, "_HA.WGS.UK10K_stand_residuals.txt", sep="")); d$res=NULL; colnames(d)=c("ID",i);m=merge(m, d, by="ID", all=T);} 
n=data.frame(m)
o=apply(n[2:ncol(n)], 2, function(x) {x[is.na(x)]=mean(x, na.rm=T);return(x)})
z=cor(o)
w=eigen(z)
62-sum(w$values[w$values>1]-1)
```

We obtain a total number of 30.58 traits.

### Effective number of SNPs

We can look up Ioanna's article (http://onlinelibrary.wiley.com/doi/10.1002/gepi.21797/suppinfo) but unfortunately the signif thresholds are calculated only for common variants at 3 different MAFs. Since we have access to the LDpruned info on the 1x, we can also calculate frequencies directly and filter. This is done using `awk`, on `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015/matrices/ldprune_indep_50_5_2/*gz`.

number of SNPs                  | p (single trait)|number of phenotypes | p (all traits)&nbsp;&nbsp;&nbsp;&nbsp;|       
--------------------------------| --------------- | ------------------- | --------------
$4,268,111$ (Ioanna's estimate) | $1.17\times10^{-08}$ | $30.58$ | $3.82\times10^{-10}$ 
$5,864,976$ (real count MAF>0.0017)|$8.52\times10^{-09}$ | $30.58$ | $2.8\times10^{-10}$


## Positive controls
### For lipid traits

```bash
file=13.22395043.peakdata.ld.annotated.assoc; awk -F, '{print "chr"$4":"$2, $6,$7, $17}' $file | awk '$NF>0.2' | cut -f1-3 -d' ' | tail -n+2 | /usr/local/bin/perl ~/CheckVariationSource_SNPID.pl | awk '$4=="+"' | while read line; do ps=$(echo $line | sed 's/.*://;s/ .*//'); rs=$(echo $line | cut -d' ' -f2); echo $ps $rs $(grep $ps $file | cut -d',' -f17 ) $(zgrep -w $rs /lustre/scratch114/projects/helic/assoc_freeze_Summer2015/replication/glc/*LDL* | awk '{print $(NF-1)}'); done | awk '$NF<0.05'
```

### For Insulin/Glucose traits

For these traits of course the nightmare begins because this data has been generated by Sanger and as such is completely disorganised. 
First, we thought we would go 80's about this:

```bash
file=9.28294231.peakdata.ld.annotated.assoc; awk -F, '{print "chr"$4":"$2, $6,$7, $17}' $file | awk '$NF>0.2' | cut -f1-3 -d' ' | tail -n+2 | /usr/local/bin/perl ~/CheckVariationSource_SNPID.pl | awk '$4=="+"' | while read line; do ps=$(echo $line | sed 's/.*://;s/ .*//'); rs=$(echo $line | cut -d' ' -f2); echo $ps $rs $(grep $ps $file | cut -d',' -f17 ); echo ========; for i in `ls /lustre/scratch114/projects/helic/assoc_freeze_Summer2015/replication/magic/*Glu*`; do echo $i; head -n1 $i; grep -w $rs $i ; echo; done; echo; echo; done | less
```

But then, merging seemed...cleaner:
```bash
# Fasting Insulin
comm -3 -2 <(cut -f1 MAGIC_Scott_et_al_FI_Jan2013.txt | sort) <(cut -f1 MAGIC_Manning_et_al_lnFastingInsulin_MainEffect.txt | sort)> only1

cat <(join -j1 -a1 <(sort -k1,1 MAGIC_Manning_et_al_lnFastingInsulin_MainEffect.txt) <(sort -k1,1 MAGIC_Scott_et_al_FI_Jan2013.txt) | awk 'BEGIN{OFS="\t"}{$11=tolower($11);$12=tolower($12);if(NF<14){$11="NA";$12=$11;$13=$11;$14=$11;$15=$11;} if($11==$3 && $12==$2){a=$12;$12=$11;$11=a;$13=-1*$13}print $1, $2","$3","$5","$7, $8","$10, $11","$12","$13","$15}') <(grep -wf only1 MAGIC_Scott_et_al_FI_Jan2013.txt | awk 'BEGIN{OFS="\t"}{print $1, "NA,NA,NA,NA\tNA,NA", $2","$3","$4","$6}') | grep -v ffect> m1

comm -3 -1 <(cut -f1 m1 | sort) <(cut -f1 MAGIC_Scott_et_al_FI_adjBMI_Jan2013.txt | sort) > only3

cat <(join -a1 -j1 -t $'\t' <(sort -k1,1 m1 | sed 's/,/ /;s/,/ /') <(sort -k1,1 MAGIC_Scott_et_al_FI_adjBMI_Jan2013.txt | awk 'BEGIN{OFS="\t"}{print $1, tolower($2)" "tolower($3)" "$4" "$6}') | awk 'BEGIN{OFS="\t"}{if(NF<8){$7="NA";$8=$7;$9=$7;$10=$7}if($2==$8 && $3==$7){$9=-1*$9;a=$7;$7=$8;$8=a}print $1, $2","$3","$4,$5,$6,$7","$8","$9","$10}') <(grep -wf only3 MAGIC_Scott_et_al_FI_adjBMI_Jan2013.txt | grep -v effe | awk 'BEGIN{OFS="\t"}{print $1, "NA,NA,NA,NA\tNA,NA\tNA,NA,NA,NA", tolower($2)","tolower($3)","$4","$6}') > m2

cat <(echo -e "SNP\tManning_FI\tManning_FIBMI\tScott_FI\tScottFIBMI") m2 > all.FI.merged

# Fasting Glucose

comm -3 -2 <(cut -f1 MAGIC_Scott_et_al_FG_Jan2013.txt | sort) <(cut -f1 MAGIC_Manning_et_al_FastingGlucose_MainEffect.txt | sort)> only1

cat <(join -j1 -a1 <(sort -k1,1 MAGIC_Manning_et_al_FastingGlucose_MainEffect.txt) <(sort -k1,1 MAGIC_Scott_et_al_FG_Jan2013.txt) | awk 'BEGIN{OFS="\t"}{$11=tolower($11);$12=tolower($12);if(NF<14){$11="NA";$12=$11;$13=$11;$14=$11;$15=$11;} if($11==$3 && $12==$2){a=$12;$12=$11;$11=a;$13=-1*$13}print $1, $2","$3","$5","$7, $8","$10, $11","$12","$13","$15}') <(grep -wf only1 MAGIC_Scott_et_al_FG_Jan2013.txt | awk 'BEGIN{OFS="\t"}{print $1, "NA,NA,NA,NA\tNA,NA", $2","$3","$4","$6}') | grep -v ffect> m1

comm -3 -1 <(cut -f1 m1 | sort) <(cut -f1 MAGIC_FastingGlucose.txt | sort) > only3

cat <(join -a1 -j1 -t $'\t' <(sort -k1,1 m1 | sed 's/,/ /;s/,/ /') <(sort -k1,1 MAGIC_FastingGlucose.txt | awk 'BEGIN{OFS="\t"}{print $1, tolower($2)" "tolower($3)" "$5" "$7}') | awk 'BEGIN{OFS="\t"}{if(NF<8){$7="NA";$8=$7;$9=$7;$10=$7}if($2==$8 && $3==$7){$9=-1*$9;a=$7;$7=$8;$8=a}print $1, $2","$3","$4,$5,$6,$7","$8","$9","$10}') <(grep -wf only3 MAGIC_FastingGlucose.txt | grep -v effe | awk 'BEGIN{OFS="\t"}{print $1, "NA,NA,NA,NA\tNA,NA\tNA,NA,NA,NA", tolower($2)","tolower($3)","$5","$7}') > m2

cat <(echo -e "SNP\tManning_FG\tManning_FGBMI\tScott_FG\tMAGIC_FG") m2 > all.FG.merged

```
(This is in the executable `README` in `/lustre/scratch114/projects/helic/assoc_freeze_Summer2015/replication/magic/`.)

#### Updated query
```bash
file=13.22395043.peakdata.ld.annotated.assoc; awk -F, '{print "chr"$4":"$2, $6,$7, $17}' $file | awk '$NF>0.7' | cut -f1-3 -d' ' | tail -n+2 | /usr/local/bin/perl /nfs/team144/ds26/tools/CheckVariationSource_SNPID.pl| awk '$4=="+" || $5=="+"' | while read line; do ps=$(echo $line | sed 's/.*://;s/ .*//'); rs=$(echo $line | cut -d' ' -f2 | tr ',' ' '); for r in $rs; do echo $ps $r $(grep $ps $file | cut -d',' -f17 ) $(zgrep -w $r /lustre/scratch114/projects/helic/assoc_freeze_Summer2015/replication/glc/*LDL*| cut -f6,8-10); done; done
```
Because different columns are merged for the 3 different runs, it's necessary to replace `$17` by `$18` in Pomak and `$21` in meta-analysis (for the latter also replace `$6` and `$7` by `$5` and `$6`).

### Querying Loz's files
Again, replace `$17` by `$18` in Pomak and `$21` in meta-analysis (for the latter also replace `$6` and `$7` by `$5` and `$6`).

```bash
file=13.22395043.peakdata.ld.annotated.assoc; grep -f <(awk -F, '{print $4, $2, $6,$7, $17}' $file | awk '$NF>0.7'| cut -d' ' -f2 | tail -n+2) /lustre/scratch113/projects/helic/Arthur/tmp/debug_metacarpa/test2/HP.2ways.LDL.withoutAge.metacarpa.out
```

### Getting `rsIDs` and HapMap information

Using Sophie's ultra-fast pattern matching using awk instead of grep.

```bash
awk 'NR==FNR{a[$1];next}{if($3 in a){print "chr"$2":"$3, $4, $5}}' t <(zcat /lustre/scratch114/projects/helic/meta_freeze_Summer2015/TG/TG.HELIC.formatted.out.gz) | /usr/local/bin/perl /nfs/team144/ds26/tools/CheckVariationSource_SNPID.pl | sed 's/chr//;s/:/ /' | sort -k1,1n -k2,2n
```

### Double checking GLC
```bash
find -type f -iname "*peakdata.ld.annotated.assoc"| while read line; do cp $line /lustre/scratch114/projects/helic/all_local_assoc; done

file=13.22395043.peakdata.ld.annotated.assoc; col=$(paste <(seq $(head -n1 $file | tr ',' '\n' | wc -l)) <(head -n1 $file | tr ',' '\n') | grep -w ld | cut -f1);LC_ALL=C zgrep -w -f <(awk -F',' -v c=$col 'NR>1 && $c>0.7{print "chr"$4":"$2, $6, $7}' $file | cut -d',' -f3 | /usr/local/bin/perl /nfs/team144/ds26/tools/CheckVariationSource_SNPID.pl | cut -f2 | tr ',' '\n') /lustre/scratch114/projects/helic/assoc_freeze_Summer2015/replication/glc/*LDL*
```